# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [ ]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

In [ ]:
# Read your api key environment variable
load_dotenv("../../resources/api_keys.env")
newsapi_key = os.getenv("NEWSAPI_API_KEY")
print(f"NewsAPI API Key Type : {type(newsapi_key)}")

In [ ]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=newsapi_key)

In [ ]:
# Fetch the Bitcoin news articles 
article_n = 20 #100 maximum allowed by free NewsAPI account
pages = int(article_n / 20)
btc_news = []

# Get 5 pages of articles (100 articles)
for i in list(range(1,pages+1)):
    articles = newsapi.get_everything(q='bitcoin', language='en', page=i)['articles']
    btc_news.append(articles)

btc_corpus = [y for x in btc_news for y in x]

print(f"Total articles about Bitcoin: {len(btc_corpus)}")

In [ ]:
# Fetch the Bitcoin news articles 
article_n = 20 #100 maximum allowed by free NewsAPI account
pages = int(article_n / 20)
eth_news = []

# Get 5 pages of articles (100 articles)
for i in list(range(1,pages+1)):
    articles = newsapi.get_everything(q='ethereum', language='en', page=i)['articles']
    eth_news.append(articles)

eth_corpus = [y for x in eth_news for y in x]

print(f"Total articles about Ethereum: {len(eth_corpus)}")

In [ ]:
# Initialize the VADER sentiment analyzer
btc_analyzer = SentimentIntensityAnalyzer()

# Create the Bitcoin sentiment scores DataFrame
btc_sentiment = []

for article in btc_corpus:
    try:
        text = article['content']
        date = article['publishedAt']
        
        sentiment = btc_analyzer.polarity_scores(text)
        compound = sentiment['compound']
        pos = sentiment['pos']
        neu = sentiment['neu']
        neg = sentiment['neg']
        
        btc_sentiment.append({
            'date' : date,
            'text' : text,
            'compound' : compound,
            'positive' : pos,
            'negative' : neg,
            'neutral' : neu
        })
        
    except AttributeError:
        print('error')
    
btc_df = pd.DataFrame(btc_sentiment)
btc_df.head()

In [ ]:
len(btc_df.index)

In [ ]:
# Initiate VADER sentiment analyzer
eth_analyzer = SentimentIntensityAnalyzer()

# Create the Ethereum sentiment scores DataFrame
eth_sentiment = []

for article in eth_corpus:
    try:
        text = article['content']
        date = article['publishedAt']
        
        sentiment = eth_analyzer.polarity_scores(text)
        compound = sentiment['compound']
        pos = sentiment['pos']
        neg = sentiment['neg']
        neu = sentiment['neu']
        
        eth_sentiment.append({
            'date' : date,
            'text' : text,
            'compound' : compound,
            'positive' : pos,
            'negative' : neg,
            'neutral' : neu
        })
        
    except AttributeError:
        pass
    
eth_df = pd.DataFrame(eth_sentiment)
eth_df.head()

In [ ]:
# Describe the Bitcoin Sentiment
btc_df.describe().round(3)

In [ ]:
# Describe the Ethereum Sentiment
eth_df.describe().round(3)

### Questions:

Q: Which coin had the highest mean positive score?

A: **BTC has the highest mean + score : 0.09**

Q: Which coin had the highest compound score?

A: **BTC has the highest compound score : 0.25**

Q. Which coin had the highest positive score?

A: **BTC has the highest positive score : 0.28**

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download('punkt')
nltk.download('wordnet')

from string import punctuation
import re

In [ ]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
custom_sw = ['char','tec','inc','say','via','bos']

In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    
    # Create a tokenized list of the words
    words = word_tokenize(re_clean)
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    lem_lower = [word.lower() for word in lem]
    
    # Remove the stop words
    tokens_sw = [word for word in lem_lower if word not in sw]
    
    # Remove custom stop words
    tokens_csw = [word for word in tokens_sw if word not in custom_sw]
    
    # Remove all words less than 3 chars long
    tokens = [word for word in tokens_csw if len(word) > 2]
    
    return tokens

In [ ]:
# Create a new tokens column for Bitcoin
btc_df['tokens'] = [tokenizer(text) for text in btc_df['text']]

# Clean up articles returning blank str for text
btc_df.replace("", float("NaN"), inplace=True)
btc_df.dropna(subset=["text"], inplace=True)
print(f"Total articles about Bitcoin remaining: {len(btc_df)}")

In [ ]:
btc_df.head()

In [ ]:
# Create a new tokens column for Ethereum
eth_df['tokens'] = [tokenizer(text) for text in eth_df['text']]

# Clean up articles returning blank str for text
eth_df.replace("", float("NaN"), inplace=True)
eth_df.dropna(subset=["text"], inplace=True)
print(f"Total articles about Ethereum remaining: {len(eth_df)}")

In [ ]:
eth_df.head()

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
btc_li = [y for x in btc_df['tokens'] for y in x]

In [ ]:
btc_grams = ngrams(btc_li, n=2)

In [ ]:
# Generate the Ethereum N-grams where N=2
eth_li = [y for x in eth_df['tokens'] for y in x]

In [ ]:
eth_grams = ngrams(eth_li, n=2)

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N : int=10):
    """Returns the top N tokens from the frequency count"""
    
    top = dict(Counter(tokens).most_common(N))
    return pd.DataFrame(list(top.items()), columns=['tokens', 'count'])

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
token_count(btc_grams,10)

In [ ]:
# Use token_count to get the top 10 words for Ethereum
token_count(eth_grams,10)

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
btc_li_bigstring = ' '.join(btc_li)
btc_wc = WordCloud(width=1600, height=800).generate(btc_li_bigstring)
plt.axis('off')
plt.imshow(btc_wc)
plt.savefig('img/btc_wc.png')

In [ ]:
# Generate the Ethereum word cloud
eth_li_bigstring = ' '.join(eth_li)
eth_wc = WordCloud(width=1500, height=800).generate(eth_li_bigstring)
plt.axis('off')
plt.imshow(eth_wc)
plt.savefig('img/eth_wc.png')

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
def replace_multiple(string, rep_list):
    for term in rep_list:
        string = string.replace(term, ' ')
    return string

rep_list = ['\r','\n','\xa0','[', ']','+','chars','-','\\']

In [ ]:
# Concatenate all of the Bitcoin text together
btc_text = ' '.join([text for text in btc_df['text']])
btc_text = replace_multiple(btc_text,rep_list)

In [ ]:
# Run the NER processor on all of the text
btc_doc = nlp(btc_text)

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
displacy.render(btc_doc, style='ent')

In [ ]:
# List all Entities
print([ent.text for ent in btc_doc.ents])

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
eth_text = ' '.join([text for text in eth_df['text']])
eth_text = replace_multiple(eth_text,rep_list)

In [ ]:
# Run the NER processor on all of the text
eth_doc = nlp(eth_text)

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
displacy.render(eth_doc, style='ent')

In [ ]:
# List all Entities
print([ent.text for ent in eth_doc.ents])

---